In [3]:
import math

In [4]:
# Example dataset
# Let'say we have a dataset two classes A and B 
# supose in a dataset of 10 elements 4 are of class A and 6 are of class B 
 

n_A = 4
n_B = 6
total = n_A + n_B

In [5]:
# let's calculate the proportion 
p_A = n_A / total
p_B = n_B / total

print("proportion of A:",p_A)
print("proportion of B:",p_B)



proportion of A: 0.4
proportion of B: 0.6


In [6]:
# Entropy calculate 

entropy = -(p_A * math.log2(p_A)) - (p_B * math.log2(p_B))
print("Entropy:",entropy)


Entropy: 0.9709505944546686


In [7]:
# gini imppurity 

gini = 1 - (p_A**2) - (p_B**2)
print("gini:",gini)

gini: 0.48


In [8]:
# information Gain
# Assuming a split on some feture divides the dataset two subset 
# subset 1:2 elements of A, of B
# subset 2:2 elements of A, of B
# Entropy and size each subset 

n_1_A , n_1_B = 0,3 
n_2_A , n_2_B = 4,3


p_1_A = n_1_A / (n_1_A + n_1_B)
p_1_B = n_1_B / (n_1_A + n_1_B)
entropy_1 =  -p_1_A * math.log2(n_1_A) - p_1_B * math.log2(n_2_B) if p_1_A and p_1_B else 0 


p_2_A = n_2_A / (n_1_A + n_1_B)
p_2_B = n_2_B / (n_1_A + n_1_B)
entropy_2 =  -p_1_A * math.log2(p_1_A) - p_1_B * math.log2(p_1_B) if p_1_A and p_1_B else 0 

# calculating information gain
information_gain = entropy - ((n_1_A + n_1_B) / total * entropy_1 + (n_2_A + n_2_B) / total * entropy_2)

print("Information Gain:", information_gain)











Information Gain: 0.9709505944546686


---


# Decsion Tress Example in Python

In [9]:
# import libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import  confusion_matrix , classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import  SimpleImputer

In [10]:
# laod titanic dataset from seaborn 

df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [11]:
# drop deck colum 
df.drop("deck", axis=1, inplace=True)

# impute missing value of age and fare using median 
imputer = SimpleImputer(strategy="median")
df[["age","fare"]] = imputer.fit_transform(df[["age","fare"]])

# impute missing values in embarked and embarked_town using mode and simple imputer 
imputer = SimpleImputer(strategy="most_frequent")
df[["embarked","embark_town"]] = imputer.fit_transform(df[["embarked","embark_town"]])

# encode categorical features to numerical ones


In [12]:
df.isnull().sum()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

In [13]:
# encode the categorical and object variables using for loop and labelencode
le = LabelEncoder()
for col in df.select_dtypes(include=['category','object']):
        df[col] = le.fit_transform(df[col])



In [14]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,1,22.0,1,0,7.2500,2,2,1,True,2,0,False
1,1,1,0,38.0,1,0,71.2833,0,0,2,False,0,1,False
2,1,3,0,26.0,0,0,7.9250,2,2,2,False,2,1,True
3,1,1,0,35.0,1,0,53.1000,2,0,2,False,2,1,False
4,0,3,1,35.0,0,0,8.0500,2,2,1,True,2,0,True


In [15]:
# split the data into X and y 
X = df.drop(['survived','alive'], axis=1)
y = df['survived']
# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)
print("Train set size: ", len(y_train))
print("Test set size: ", len(y_test))


Train set size:  712
Test set size:  179


In [16]:
# create the model and train the model with pred
model = DecisionTreeClassifier(criterion='entropy',max_depth=100, random_state=42)
model.fit(X_train, y_train)
#predict the model 
y_pred = model.predict(X_test)


In [17]:
#evaluate the model 
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print("Classification Report: \n", classification_report(y_test, y_pred))

Confusion Matrix: 
 [[82 23]
 [21 53]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.80      0.78      0.79       105
           1       0.70      0.72      0.71        74

    accuracy                           0.75       179
   macro avg       0.75      0.75      0.75       179
weighted avg       0.76      0.75      0.75       179



In [18]:
# save the decision tree classification 
from sklearn.tree import export_graphviz
export_graphviz = export_graphviz(model, out_file='./saved_models/decision_tree04.dot', feature_names=X.columns, filled=True, rounded=True, special_characters=True)